In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import time

# Web-Scrapping

In [2]:
url = 'https://myanimelist.net/topanime.php'
limit = 50
params = [{'limit':limit*i} for i in range(0,20)]
soup = []

for i in range(0,20):
    page = requests.get(url, params=params[i])

    if page.status_code == 200:
        soup.append(BeautifulSoup(page.text, 'html'))
    else:
        print(f"Request failed")

In [3]:
## soup contains html of 21 pages of the Top Rated Anime Series of all Time (Top 1000 Anime)
len(soup)

20

### Target Table
###### < table border="0" cellpadding="0" cellspacing="0" width="100%" class="top-ranking-table" >

In [4]:
#### Fetching Table header

table = soup[0].find('table',class_='top-ranking-table')
header_element = table.find('tr', class_ = 'table-header')
header = header_element.find_all('td')

title = []
for head in header:
    title.append(head.text)

title = title[:3]

In [5]:
title.extend(['popularity','Members','Type', 'Episodes', 'Genre', 'Demographic','Start Date', 'End Date',  'Image', 'Link'])

### Creation of Data-Frame

In [6]:
df = pd.DataFrame(columns = title)
df

,Rank,Title,Score,popularity,Members,Type,Episodes,Genre,Demographic,Start Date,End Date,Image,Link


### Fetching Anime Data

In [10]:
Row_Master = []

In [13]:
for i in tqdm(range(0,20), desc=f"-------------Processing Page----------", unit="item"):

    table = soup[i].find('table',class_='top-ranking-table')
    row_data = soup[i].find_all('tr', class_='ranking-list')

    for j in tqdm(range(len(row_data)), desc="Processing Anime", unit="item"):

        ### Initializing default values in case Data is missing
        rank = 0
        title = ''
        score = 0
        popularity=0
        members=0
        anime_type=''
        episodes=0
        genre=''
        demographic=''
        start_date=''
        end_date=''
        img_value=''
        href_value=''


        rd = row_data[j].find_all('td')

        rank_tag = rd[0].find('span', class_ = 'lightLink')
        if rank_tag:
            rank =rank_tag.text
            rank = int(rank)

        anchor_tag = rd[1].find('a')
        if anchor_tag:
            href_value = anchor_tag.get('href')



        img_tag = rd[1].find('img')
        if img_tag:
            img_value = img_tag.get('data-srcset').split(", ")
            img_value = [img[:-3] for img in img_value]
            img_value = img_value[1]

        name_div = rd[1].find('div', class_='di-ib')
        if name_div:
                title = name_div.find('a',class_='hoverinfo_trigger').text
                title = title.replace('"', '')


        detail_div = rd[1].find('div', class_='information')
        if detail_div:
                details = detail_div.text.split('\n')
                details = [det.strip() for det in details if det.strip()!='']



        score_div = rd[2].find('span')
        if score_div:
                score =score_div.text
                score = float(score)

        res = requests.get(href_value)
        if res.status_code == 200:
            anime_soup = BeautifulSoup(res.text, 'html')
            table = anime_soup.find('table')
            table_row = table.find('tr')
            pop_div = table_row.find('div', class_='stats-block po-r clearfix')

            strong = pop_div.find('span', class_='numbers popularity')
            strong = strong.find_all('strong')
            popularity = strong[0].text[1:]
            popularity = int(popularity)

            left_div = anime_soup.find('div', class_='leftside')
            info_div = left_div.find_all('div', class_='spaceit_pad')

            for info in info_div:

                text = info.find('span').text
                if text == 'Demographic:':
                    demographic = info.find_all('span')[1].text
                elif text == 'Genres:' or text == 'Genre:':
                    genre = info.find_all('span')
                    genres = []
                    for gen in genre:
                        genres.append(gen.text)
                    genre = [gen.strip() for gen in genres]
                    genre=genre[1:]

        members = details[2]
        members = ''.join(filter(str.isdigit, members))
        members = int(members)


        anime_type_eps = details[0].split(' ')
        anime_type = anime_type_eps[0]
        episodes = anime_type_eps[1][1:]
        if episodes != '?':
            episodes = int(episodes)

        if "to" in details[1]:
            date_string = details[1]
            start_date, end_date = date_string.split(' to ')
        else:
            date_string = details[1]
            if len(date_string.split(' - ')) == 1:
                start_date,end_date = date_string.split(' -')
                start_date = datetime.strptime(start_date, '%b %Y').date()

            else:
                start_date, end_date = date_string.split(' - ')
                start_date = datetime.strptime(start_date, '%b %Y').date()
                end_date = datetime.strptime(end_date, '%b %Y').date()

        new_list = [rank,title,score,popularity, members,anime_type, episodes, genre, demographic,start_date, end_date, img_value, href_value]
        Row_Master.append(new_list)
        time.sleep(3)
    print(new_list)

-------------Processing Page----------:   5%|▌         | 1/20 [03:26<1:05:27, 206.72s/item]

[50, 'Mushishi Zoku Shou 2nd Season', 8.73, 896, 259545, 'TV', 10, ['Adventure', 'Mystery', 'Slice of Life', 'Supernatural'], 'Seinen', datetime.date(2014, 10, 1), datetime.date(2014, 12, 1), 'https://cdn.myanimelist.net/r/100x140/images/anime/9/68095.jpg?s=847f720879a312ca3ac84fd8327b174e', 'https://myanimelist.net/anime/24701/Mushishi_Zoku_Shou_2nd_Season']



-------------Processing Page----------:  10%|█         | 2/20 [06:50<1:01:31, 205.06s/item]

[100, 'Ookami Kodomo no Ame to Yuki', 8.57, 225, 797107, 'Movie', 1, ['Award Winning', 'Fantasy', 'Slice of Life'], '', datetime.date(2012, 7, 1), datetime.date(2012, 7, 1), 'https://cdn.myanimelist.net/r/100x140/images/anime/9/35721.jpg?s=8447e90edf5c69af409b8483f91a72e9', 'https://myanimelist.net/anime/12355/Ookami_Kodomo_no_Ame_to_Yuki']



-------------Processing Page----------:  15%|█▌        | 3/20 [10:13<57:49, 204.10s/item]  

...

[900, 'Michiko to Hatchin', 7.83, 1159, 197414, 'TV', 22, ['Action', 'Adventure'], '', datetime.date(2008, 10, 1), datetime.date(2009, 3, 1), 'https://cdn.myanimelist.net/r/100x140/images/anime/10/59535.jpg?s=c8ed17a081b56853c3c9a0b1d3e8fa37', 'https://myanimelist.net/anime/4087/Michiko_to_Hatchin']



-------------Processing Page----------:  95%|█████████▌| 19/20 [1:05:10<03:26, 206.84s/item]

[950, 'Detective Conan Movie 07: Crossroad in the Ancient Capital', 7.8, 2984, 50930, 'Movie', 1, ['Adventure', 'Comedy', 'Mystery'], 'Shounen', datetime.date(2003, 4, 1), datetime.date(2003, 4, 1), 'https://cdn.myanimelist.net/r/100x140/images/anime/10/20984.jpg?s=a714a1dabf5d362aab7c026363b68c63', 'https://myanimelist.net/anime/1366/Detective_Conan_Movie_07__Crossroad_in_the_Ancient_Capital']



-------------Processing Page----------: 100%|██████████| 20/20 [1:08:41<00:00, 206.08s/item]

[1000, 'Sonny Boy', 7.78, 782, 294213, 'TV', 12, ['Award Winning', 'Mystery'], '', datetime.date(2021, 7, 1), datetime.date(2021, 10, 1), 'https://cdn.myanimelist.net/r/100x140/images/anime/1509/117149.jpg?s=109b728e658c177da07ad2c897d45fc6', 'https://myanimelist.net/anime/48849/Sonny_Boy']


In [14]:
Row_Master

[[1,
  'Fullmetal Alchemist: Brotherhood',
  9.1,
  3,
  3247131,
  'TV',
  64,
  ['Action', 'Adventure', 'Drama', 'Fantasy'],
  'Shounen',
  datetime.date(2009, 4, 1),
  datetime.date(2010, 7, 1),
  'https://cdn.myanimelist.net/r/100x140/images/anime/1208/94745.jpg?s=5ec18639199f2c60b60009f34222228d',
  'https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood'],
 [2,
  'Steins;Gate',
  9.07,
  13,
  2495249,
  'TV',
  24,
  ['Drama', 'Sci-Fi', 'Suspense'],
  '',
  datetime.date(2011, 4, 1),
  datetime.date(2011, 9, 1),
  'https://cdn.myanimelist.net/r/100x140/images/anime/1935/127974.jpg?s=a0c19821033ace3af10310a253a9e85b',
  'https://myanimelist.net/anime/9253/Steins_Gate'],
 [3,
  'Gintama°',
  9.06,
  335,
  611640,
  'TV',
  51,
  ['Action', 'Comedy', 'Sci-Fi'],
  'Shounen',
  datetime.date(2015, 4, 1),
  datetime.date(2016, 3, 1),
  'https://cdn.myanimelist.net/r/100x140/images/anime/3/72078.jpg?s=bb26d45bb75e13601e1b8029f717c2b0',
  'https://myanimelist.net/anime/289

In [15]:
rm = Row_Master.copy()

In [27]:
df.drop(df.index, inplace=True)

In [16]:
for row in Row_Master:
    new_df = pd.DataFrame([row], columns=df.columns)
    df = pd.concat([df, new_df], ignore_index=True,axis=0)

In [17]:
df.tail()

,Rank,Title,Score,popularity,Members,Type,Episodes,Genre,Demographic,Start Date,End Date,Image,Link
995,996,Mind Game,7.78,2229,84124,Movie,1,"[Adventure, Avant Garde, Award Winning, Comedy...",,2004-08-01,2004-08-01,https://cdn.myanimelist.net/r/100x140/images/a...,https://myanimelist.net/anime/875/Mind_Game
996,997,Saiyuuki Gaiden,7.78,5476,13542,OVA,3,"[Action, Adventure, Drama, Fantasy]",Shounen,2011-03-01,2011-11-01,https://cdn.myanimelist.net/r/100x140/images/a...,https://myanimelist.net/anime/9088/Saiyuuki_Ga...
997,998,Seitokai Yakuindomo Movie 2,7.78,3852,31175,Movie,1,[Comedy],Shounen,2021-01-01,2021-01-01,https://cdn.myanimelist.net/r/100x140/images/a...,https://myanimelist.net/anime/40814/Seitokai_Y...
998,999,Shin Kidou Senki Gundam Wing: Endless Waltz Sp...,7.78,2746,59142,Movie,1,"[Action, Drama, Sci-Fi]",,1998-08-01,1998-08-01,https://cdn.myanimelist.net/r/100x140/images/a...,https://myanimelist.net/anime/2273/Shin_Kidou_...
999,1000,Sonny Boy,7.78,782,294213,TV,12,"[Award Winning, Mystery]",,2021-07-01,2021-10-01,https://cdn.myanimelist.net/r/100x140/images/a...,https://myanimelist.net/anime/48849/Sonny_Boy


In [18]:
df.shape

(1000, 13)

In [22]:
df.iloc[999]

Rank                                                        1000
Title                                                  Sonny Boy
Score                                                       7.78
popularity                                                   782
Members                                                   294213
Type                                                          TV
Episodes                                                      12
Genre                                   [Award Winning, Mystery]
Demographic                                                     
Start Date                                            2021-07-01
End Date                                              2021-10-01
Image          https://cdn.myanimelist.net/r/100x140/images/a...
Link               https://myanimelist.net/anime/48849/Sonny_Boy
Name: 999, dtype: object

In [23]:
df.to_csv('Anime_Data.csv', index=False)